In [ ]:
import math
(8*(math.pi-3))/(3*math.pi*(4-math.pi))


In [1]:
import torch
x = torch.Tensor([2,3,.99,0.5])
y = torch.erfinv(x.to('mps'))
y

RuntimeError: Failed to compile library: program_source:8:19: error: use of undeclared identifier 'erf'
  output[index] = erf(input[index]);
                  ^


In [1]:
import torch
x = torch.tensor([1, 0.9999999403953 ,0.5, 0.3, 0.2, -0.0000000000001, -0.45, .880, .880, -0.9999999403953552,-1], dtype=torch.float32)
y_mps = torch.erfinv(x.to("mps"))
y_cpu = torch.erfinv(x.to("cpu"))
print(y_mps)
print(y_cpu)
print(y_mps.dtype)

tensor([        inf,  3.8325e+00,  4.7694e-01,  2.7246e-01,  1.7914e-01,
        -8.8623e-14, -4.2268e-01,  1.0994e+00,  1.0994e+00, -3.8325e+00,
               -inf], device='mps:0')
tensor([        inf,  3.8325e+00,  4.7694e-01,  2.7246e-01,  1.7914e-01,
        -8.8623e-14, -4.2268e-01,  1.0994e+00,  1.0994e+00, -3.8325e+00,
               -inf])
torch.float32


In [2]:
import torch
x = torch.arange(-1, 1, 1e-8) # default cpu tensor
#measure CPU compute time by calling torch.erfinv
time = %timeit -o -q torch.erfinv(x)
cpu_time = time.average
print("CPU torch.erfinv time: ", cpu_time)

x = x.to("mps")
# measure MPS compute time
time = %timeit -o -q  torch.erfinv(x)
mps_time = time.average
print("MPS torch.erfinv time: ", mps_time)
print(f"MPS torch.erfinv is {cpu_time/mps_time*100} percent faster than CPU torch.erfinv")

# compute MSE between MPS and CPU torch.erfinv
x = x.to("cpu")
print("Done convert x to cpu device")
y_cpu = torch.erf(x)
print("Done computing cpu torch.erfinv")
x = x.to("mps")
print("Done convert x to mps device")
y_mps = torch.erf(x)
print("Done computing mps torch.erfinv")

y_mps = y_mps.to("cpu")
print("computing mask")
mask = torch.isfinite(y_cpu) & torch.isfinite(y_mps.to("cpu"))
y_mps = y_mps[mask]
y_cpu = y_cpu[mask]
x = x[mask]
print
mse = torch.square(y_cpu - y_mps).mean()
print("MSE between MPS and CPU torch.erfinv: ", mse)
diff = torch.abs(y_cpu - y_mps)

print("Largest difference")
print(f"x:  {x[torch.argmax(diff)]}, y_cpu: {y_cpu[torch.argmax(diff)]}, y_mps: {y_mps[torch.argmax(diff)]} , diff = {y_cpu[torch.argmax(diff)] - y_mps[torch.argmax(diff)]}")



CPU torch.erfinv time:  2.702730720139308
MPS torch.erfinv time:  0.009846114047187646
MPS torch.erfinv is 27449.719830447135 percent faster than CPU torch.erfinv
Done convert x to cpu device
Done computing cpu torch.erfinv
Done convert x to mps device
Done computing mps torch.erfinv
computing mask
MSE between MPS and CPU torch.erfinv:  tensor(2.6448e-15)
Largest difference
x:  -0.9999987483024597, y_cpu: -0.8427003026008606, y_mps: -0.8427001237869263 , diff = -1.7881393432617188e-07


```
CPU torch.erfinv time:  2.646950892857473
MPS torch.erfinv time:  0.13262139940000323
MPS torch.erfinv is 1995.8701271684884 percent faster than CPU torch.erfinv
Done convert x to cpu device
Done computing cpu torch.erfinv
Done convert x to mps device
Done computing mps torch.erfinv
```

In [2]:
import numpy as np

def calc_erfinv(y):
    # a = np.array([0.886226899, -1.645349621, 0.914624893, -0.140543331])
    # b = np.array([-2.118377725, 1.442710462, -0.329097515, 0.012229801])
    # c = np.array([-1.970840454, -1.624906493, 3.429567803, 1.641345311])
    # d = np.array([3.543889200, 1.637067800])
    # CENTRAL_RANGE = 0.7

    # y_abs = np.abs(y)
    # if y_abs > 1.0: 
    #     return np.nan
    # if y_abs == 1.0: 
    #     return np.sign(y) * np.inf
    # if y_abs <= CENTRAL_RANGE:
    #     z = y * y
    #     num = (((a[3]*z + a[2])*z + a[1])*z + a[0])
    #     dem = ((((b[3]*z + b[2])*z + b[1])*z + b[0]) * z + 1.0)
    #     x = y * num / dem
    # else:
    #     print("going to else statement")
    #     z = np.sqrt(-np.log((1.0 - y_abs) / 2.0))
    #     num = ((c[3]*z + c[2])*z + c[1]) * z + c[0]
    #     dem = (d[1]*z + d[0])*z + 1.0
    #     x = np.sign(y) * num / dem
    
    # before_newton = x
    #y = 0.79
    x = 0.8219115
    print(f"initial x:{x}, y:{y}")
    before_newton = x
    # Two steps of Newton-Raphson correction 
    print("erf(x)", np.math.erf(x))
    print("erf(x) - y",(np.math.erf(x) - y))
    for _ in range(1):
        x = x - (np.math.erf(x) - y) / ((2.0 / np.sqrt(np.pi)) * np.exp(-x*x))

    print("diff between before and after newton", x - before_newton)
    return x


In [4]:
# create dtype = np.float32
calc_erfinv(np.array([0.75491], dtype=np.float32))

initial x:0.8219115, y:[0.75491]
erf(x) 0.7549100823820893
erf(x) - y [1.1920929e-07]
diff between before and after newton [-1.7881393e-07]


array([0.82191133], dtype=float32)

In [1]:
import torch
x = torch.tensor([0.75491], dtype=torch.float32)
y = torch.erfinv(x.to('mps'))
y


tensor([0.8219], device='mps:0')

In [6]:
y.to('cpu').numpy() - np.array([0.7900000914423133], dtype=np.float32)

array([-5.9604645e-08], dtype=float32)

In [7]:
x2 = torch.tensor([0.75491, 0.7900000144423], dtype=torch.float32)
print(torch.erf(x2.to('mps')))
print(torch.erf(x2.to('cpu')))
print(torch.erf(x2.to('mps')).to('cpu').numpy() - torch.erf(x2.to('cpu')).numpy())

tensor([0.7143, 0.7361], device='mps:0')
tensor([0.7143, 0.7361])
[5.9604645e-08 5.9604645e-08]


In [3]:
torch.exp(-x**2)

tensor([0.3679, 0.3679, 0.7788, 0.9139, 0.9608, 1.0000, 0.8167, 0.4610, 0.4610,
        0.3679, 0.3679])

In [5]:
torch.erf(x) - torch.erf(x.to('cpu'))

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [1]:
import matplotlib.pyplot as plt
# plot y_mps and x
x = torch.arange(-1, 1, 0.000001)
x = x.to("mps")
y_mps = torch.erfinv(x)
plt.plot(x.to("cpu"), y_mps.to("cpu"))
plt.xlabel("x")
plt.ylabel("y")
plt.title("y = torch.erfinv(x)")
plt.show()

NameError: name 'torch' is not defined

In [ ]:
import torch
shape = (1,10)
#cpu_x = torch.randint(0, 2, shape, device='cpu', dtype=torch.int32, requires_grad=False)
# create cpu_x as a dtype of int16
cpu_x = torch.randfloat(0, 1.0, shape, device='cpu', dtype=torch.float32, requires_grad=False)
#cpu_x = torch.Tensor

mps_x = cpu_x.to('mps')

cpu_y = torch.erfinv(cpu_x)
mps_y = torch.erfinv(mps_x)

#print("MSE between MPS and CPU torch.erfinv: ", torch.square(cpu_y - mps_y.to('cpu')).mean())
mps_y


In [ ]:
import plotly.express as px

fig = px.line(x=x.cpu(), y=y_metal.cpu(), title="erfinv(x) on MPS device")

# show graph
fig.show()

In [ ]:
    def test_unary_ops():
        def helper(shape, op):
            for dtypef in [torch.float32]:
                cpu_x = torch.randn(shape, device='cpu', dtype=dtypef, requires_grad=False)
                mps_x = cpu_x.detach().clone().to('mps')
                self.assertEqual(op(cpu_x), op(mps_x))

            for dtypei in [torch.int32, torch.int16]:
                cpu_x = torch.randint(0, 1000, shape, device='cpu', dtype=dtypei, requires_grad=False)
                mps_x = cpu_x.to('mps')
                self.assertEqual(op(cpu_x), op(mps_x), rtol=1e-4, atol=1e-4)
        helper((2, 8, 3, 5), torch.erfinv)
    
    test_unary_ops()

In [ ]:
no_frac = torch.frac(x)
no_frac